In [1]:
import numpy as np
from sklearn import datasets
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence

In [2]:
#                    Description of the dataset
#  data : {ndarray, dataframe} of shape (150, 4)
#             The data matrix. If `as_frame=True`, `data` will be a pandas
#             DataFrame.
#         target: {ndarray, Series} of shape (150,)
#             The classification target. If `as_frame=True`, `target` will be
#             a pandas Series.
#         feature_names: list
#             The names of the dataset columns.
#         target_names: list
#             The names of target classes.
#         frame: DataFrame of shape (150, 5)
#             Only present when `as_frame=True`. DataFrame with `data` and
#             `target`.

In [3]:
# Creating an output array
def target_array(raw_target):
    target = []
    for i in range(len(raw_target)):
        if raw_target[i] == 0:
            target.append([1, 0, 0])
        elif raw_target[i] == 1:
            target.append([0, 1, 0])
        else:
            target.append([0, 0, 1])
    return np.array(target, dtype="uint8")
    

In [4]:
# Loading the Iris dataset
iris = datasets.load_iris()

data = iris.data
target = target_array(iris.target)

# Mixing data
# np.random.seed(1)
# np.random.shuffle(data)
# np.random.seed(1)
# np.random.shuffle(target)


rs1 = RandomState(MT19937(SeedSequence(123456789)))
rs1.shuffle(data)
rs2 = RandomState(MT19937(SeedSequence(123456789)))
rs2.shuffle(target)


In [5]:
# RELU
# Activation function
def activation_func(x): 
    z = (x > 0) * x     # returns x if x > 0
    return z            # return 0 otherwise


# Derivative of the activation function
def deriv_activation_func(x):
    z = x > 0
    return z            # returns 1 for input > 0

0 - input layer  
1 - hidden layer  
2 - output layer  
weights_1_to_2 - Weights linking the hidden layer and the output layer

In [6]:
# Neural network learning function
def neural_network_training(weights_0_to_1,
                            weights_1_to_2,
                            data,
                            target,
                            number_training_data,
                            epochs=5,
                            alpha=0.5):
    epoch_print_step = 100
    for epoch in range(epochs):
        if epoch % epoch_print_step == 0:
            print(f"\nepoch: {epoch + 1}")
        for i in range(number_training_data):
            # Forward propagation
            layer_0 = data[i:i + 1]
            layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
            layer_2 = np.dot(layer_1, weights_1_to_2)

            # Error calculation
            layer_2_delta = layer_2 - target[i:i + 1]
            layer_1_delta = layer_2_delta.dot(weights_1_to_2.T) * deriv_activation_func(layer_1)
            
            # Backpropagation
            weights_1_to_2 -= alpha * layer_1.T.dot(layer_2_delta)
            weights_0_to_1 -= alpha * layer_0.T.dot(layer_1_delta)
            
            # Output of a part of the data
            if epoch % epoch_print_step == 0 and i >= 10 and i <= 13:
                print(f"{np.argmax(layer_2) == np.argmax(target[i])} Error: {np.sum((layer_2 - target[i]) ** 2):.5f}, out = {layer_2}, target = {target[i]}")
  
    return weights_0_to_1, weights_1_to_2

In [7]:
# Declaring parameters for a neural network
# epochs = 1000
epochs = 201
alpha = 0.002

number_test_data = 30
len_data = len(data)
number_training_data = len_data - number_test_data
layer_1_size = 6

np.random.seed(1)
weights_0_to_1 = np.random.rand(len(data[0]), layer_1_size)
weights_1_to_2 = np.random.rand(layer_1_size, len(target[0]))

In [8]:
# Neural network training
weights_0_to_1, weights_1_to_2 = neural_network_training(weights_0_to_1,
                                                         weights_1_to_2,
                                                         data,
                                                         target,
                                                         number_training_data,
                                                         epochs,
                                                         alpha)


epoch: 1
False Error: 2.90231, out = [[0.14087913 0.61379879 1.65327451]], target = [0 1 0]
False Error: 1.74543, out = [[-0.01617605  0.37124343  1.16182256]], target = [0 1 0]
False Error: 2.90079, out = [[-0.38354142 -0.11704163  0.98636118]], target = [1 0 0]
True Error: 2.01745, out = [[0.62075165 1.24214462 1.29865531]], target = [0 0 1]

epoch: 101
True Error: 0.45220, out = [[0.18610408 0.46101934 0.35646169]], target = [0 1 0]
True Error: 0.54897, out = [[0.32808525 0.36053528 0.18003811]], target = [0 1 0]
True Error: 0.09188, out = [[ 0.93387562  0.2500973  -0.15799437]], target = [1 0 0]
False Error: 0.58421, out = [[-0.20228493  0.58052842  0.54581998]], target = [0 0 1]

epoch: 201
True Error: 0.49168, out = [[0.18683411 0.43825735 0.37579241]], target = [0 1 0]
True Error: 0.58237, out = [[0.33035355 0.34048395 0.1956455 ]], target = [0 1 0]
True Error: 0.08293, out = [[ 0.93935625  0.23747158 -0.1511992 ]], target = [1 0 0]
True Error: 0.51457, out = [[-0.19448204  0.5

In [9]:
def сhecking_accuracy(data, target):    
    if np.argmax(data) == np.argmax(target):
        print("True")
        return
    print(f"False: {data}, {target}")

In [10]:
def neural_network_test(weights_0_to_1,
                        weights_1_to_2,
                        data,
                        target,
                        number_test_data,
                        len_data):
    # print(len_data - number_test_data, len_data)
    for i in range(len_data - number_test_data, len_data):
        layer_0 = data[i:i + 1]
        layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
        layer_2 = np.dot(layer_1, weights_1_to_2)
        # error = np.sum((layer_2 - target[i]) ** 2)
        
        сhecking_accuracy(layer_2, target[i])
        # print(f"target = out: {np.argmax(target[i]) == np.argmax(layer_2)}\terror = {error:.4f}")
        # print(f"target = {target[i]}, out = {layer_2},\terror = {error:.4f}")

In [11]:
neural_network_test(weights_0_to_1,
                    weights_1_to_2,
                    data,
                    target,
                    number_test_data,
                    len_data)

True
True
False: [[0.08897259 0.37922361 0.41013502]], [0 1 0]
True
True
True
True
False: [[-0.12509756  0.44421427  0.57005466]], [0 1 0]
True
False: [[0.03676983 0.45859428 0.52365791]], [0 1 0]
True
True
True
True
True
False: [[0.15112464 0.43979363 0.45690389]], [0 1 0]
True
True
True
True
True
False: [[0.08052462 0.3026604  0.32361188]], [0 1 0]
True
False: [[0.09333004 0.41395115 0.44917475]], [0 1 0]
True
False: [[0.02019161 0.41692672 0.48125266]], [0 1 0]
True
True
False: [[-0.05470459  0.42140907  0.51578541]], [0 1 0]
True


In [12]:
def neural_network(weights_0_to_1,
                   weights_1_to_2,
                   data):
    layer_0 = data
    layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
    layer_2 = np.dot(layer_1, weights_1_to_2)    
    return layer_2

In [13]:
for i in range(20):
    number = np.random.randint(0, 150)
    prediction = neural_network(weights_0_to_1,
                                weights_1_to_2,
                                data[number])
    сhecking_accuracy(prediction, target[number])

True
False: [0.15112464 0.43979363 0.45690389], [0 1 0]
True
True
False: [0.09484111 0.36329519 0.38915378], [0 1 0]
False: [0.11314198 0.37138107 0.39148717], [0 1 0]
True
True
True
True
True
True
True
True
True
True
False: [0.09333004 0.41395115 0.44917475], [0 1 0]
True
True
True
